In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db_user = ""
db_password = ""
db_host = "" 
db_port = ""  
db_name = ""
connection = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection)

In [ ]:
query= (""" SELECT 
    geom_data.layerid AS layerid, geom_data.geojson , 
    COALESCE(datavalue, 0) AS datavalue, 
    COALESCE(ni, 0) AS ni, 
    COALESCE(ns, 0) AS ns, 
    COALESCE(tb, 0) AS tb, 
    COALESCE(tc, 0) AS tc, 
    COALESCE(gm, 0) AS gm, 
    COALESCE(gf, 0) AS gf, 
    COALESCE(f1, 0) AS f1, 
    COALESCE(f2, 0) AS f2, 
    COALESCE(f3, 0) AS f3, 
    COALESCE(f4, 0) AS f4, 
    COALESCE(f5, 0) AS f5, 
    COALESCE(f6, 0) AS f6
FROM geom_data
LEFT JOIN (
    SELECT 
        layerid, 
        SUM(datavalue) AS datavalue, 
        SUM(Ni) AS ni, 
        SUM(Ns) AS ns, 
        SUM(Tb) AS tb, 
        SUM(Tc) AS tc, 
        SUM(Gm) AS gm, 
        SUM(Gf) AS gf, 
        SUM(F1) AS f1, 
        SUM(F2) AS f2, 
        SUM(F3) AS f3, 
        SUM(F4) AS f4, 
        SUM(F5) AS f5, 
        SUM(F6) AS f6 
    FROM movements 
    GROUP BY layerid
) AS move 
ON move.layerid = geom_data.layerid;
""")

try:
    with engine.connect() as connection:
        chunks = pd.read_sql(query, connection, chunksize=1000) 
        df = pd.concat(chunks, ignore_index=True)  
    print(df)
except Exception as e:
    print(f"Errore durante l'esecuzione della query: {e}")

In [4]:

import folium
import geopandas as gpd
from shapely.geometry import shape
df["id"] = df["geojson"].apply(lambda geojson: geojson['id'])
geo = gpd.GeoDataFrame(df, geometry=df['geojson'].apply(lambda geojson: shape(geojson['geometry'])))
geojson_data =  {
    "type": "FeatureCollection",
    "features": [{"type": "Feature", "geometry": geom} if geom.get("type") != "Feature" else geom for geom in df['geojson']]
}
df.drop(columns=["geojson","layerid","id"], inplace=True )
geo=geo[geo["datavalue"]!=0]
features=df.columns.tolist()

In [5]:
def style_function_generator(colormap, value_dict):
    def style_function(feature):
        id_value = feature["id"]
        return {
            "fillColor": colormap(value_dict[id_value]) if id_value in value_dict else "#000000",
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.9,
        }
    return style_function

Rappresentazione grafica partenze per layerid

In [ ]:
from branca.colormap import linear
m =  folium.Map()
bounds =geo.total_bounds 
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
dictlist=[]
colorlist=["YlGn_09", "BuGn_09", "PuBu_09", "OrRd_09", 
    "PuRd_09", "Blues_09", "Greens_09", "Oranges_09", 
    "Purples_09", "Reds_09", "BuPu_09", "GnBu_09", "YlOrRd_09"
]
dizionario = {key: colorlist[i] for i, key in enumerate(features)}

for f in features:
    
    value_dict=geo.set_index("id")[f].to_dict()
    colormap = getattr(linear, dizionario[f]).scale(geo.select_dtypes(include='number').min().min(), geo.select_dtypes(include='number').max().max())
    print(f"color map per la feature {f}")
    display(colormap)
    folium.GeoJson(
    geojson_data,
    name="partenze "+ f,
    style_function=style_function_generator(colormap, value_dict),
    ).add_to(m)

folium.LayerControl().add_to(m)    
m

In [ ]:
query= (""" SELECT 
    geom_data.layerid AS toid, geom_data.geojson , 
    COALESCE(datavalue, 0) AS datavalue, 
    COALESCE(ni, 0) AS ni, 
    COALESCE(ns, 0) AS ns, 
    COALESCE(tb, 0) AS tb, 
    COALESCE(tc, 0) AS tc, 
    COALESCE(gm, 0) AS gm, 
    COALESCE(gf, 0) AS gf, 
    COALESCE(f1, 0) AS f1, 
    COALESCE(f2, 0) AS f2, 
    COALESCE(f3, 0) AS f3, 
    COALESCE(f4, 0) AS f4, 
    COALESCE(f5, 0) AS f5, 
    COALESCE(f6, 0) AS f6
FROM geom_data
LEFT JOIN (
    SELECT 
        toid, 
        SUM(datavalue) AS datavalue, 
        SUM(Ni) AS ni, 
        SUM(Ns) AS ns, 
        SUM(Tb) AS tb, 
        SUM(Tc) AS tc, 
        SUM(Gm) AS gm, 
        SUM(Gf) AS gf, 
        SUM(F1) AS f1, 
        SUM(F2) AS f2, 
        SUM(F3) AS f3, 
        SUM(F4) AS f4, 
        SUM(F5) AS f5, 
        SUM(F6) AS f6 
    FROM movements  
    GROUP BY toid
) AS move 
ON move.toid = geom_data.layerid;
""")

try:
    with engine.connect() as connection:
        chunks = pd.read_sql(query, connection, chunksize=1000)  
        df = pd.concat(chunks, ignore_index=True) 
    print(df)
except Exception as e:
    print(f"Errore durante l'esecuzione della query: {e}")
connection.close()

In [9]:
df["id"] = df["geojson"].apply(lambda geojson: geojson['id'])
geo = gpd.GeoDataFrame(df, geometry=df['geojson'].apply(lambda geojson: shape(geojson['geometry'])))
df.drop(columns=["geojson","toid","id"], inplace=True )
geo=geo[geo["datavalue"]!=0]
features=df.columns.tolist()


Rappresentazione grafica arrivi per toid

In [ ]:
m =  folium.Map()

m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])


for f in features:
    value_dict=geo.set_index("id")[f].to_dict()
    colormap = getattr(linear, dizionario[f]).scale(geo.select_dtypes(include='number').min().min(), geo.select_dtypes(include='number').max().max())
    print(f"color map per la feature {f}")
    display(colormap)
    folium.GeoJson(
    geojson_data,
    name="arrivi "+ f,

    style_function=style_function_generator(colormap, value_dict),
    ).add_to(m)

folium.LayerControl().add_to(m)    
m